<figure>
  <IMG SRC="Logo.png"  WIDTH=150 ALIGN="right">
</figure>

## Projections of mean sea level and tides<br>
### Six Dutch stations
<b>Prepared by: H.G. Voortman</b>


<figure>
    <IMG SRC="https://www.python.org/static/community_logos/python-powered-w-200x80.png"  WIDTH=100 ALIGN="right">
</figure>

#### Description

#### References
- Provided in report

#### Packages

In [1]:
# General packages
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3 as sq
import numpy as np
#import scipy.signal as signal
#import datetime as dt
import hvec_stat.general_fit as gf
#import hvec_stat.gof as gof
import hvec_stat.support as sup
import models as mdl

In [2]:
# Settings
plt.rcParams['axes.grid'] = True
figsize = (20, 18)

#### Connect databases and import data

##### Processed data Rijkswaterstaat

In [3]:
# Connect database
conn_str = os.getenv('DATAPATH') + 'RWS_processed.db'
cnxn = sq.connect(conn_str, detect_types = True)

In [4]:
pd.read_sql('SELECT * FROM sqlite_master', cnxn)

,type,name,tbl_name,rootpage,sql
0,table,const_yr,const_yr,2,"CREATE TABLE ""const_yr"" (\n""naam"" TEXT,\n ""le..."
1,index,ix_const_yr_naam_level_1,const_yr,3,"CREATE INDEX ""ix_const_yr_naam_level_1""ON ""con..."


In [5]:
# Read table with observed water levels; complete years only
sql = (
    "SELECT * "
    "FROM 'const_yr' "
    "WHERE naam IN ('Delfzijl', 'Harlingen', "
    "'Den Helder', 'IJmuiden',  "
    "'Hoek van Holland', 'Vlissingen') "
    "AND (count>650) "
#    "AND (set == 'Ftested3') "
)
df = pd.read_sql(sql, cnxn) #.groupby('YEAR').mean()

In [6]:
cnxn.close()

In [7]:
df.columns = df.columns.str.replace('_ampl', '')
df.columns

Index(['naam', 'level_1', 'z0', 'zmean', 'count', 'M2', 'S2', 'Rsq_adj',
       'MHWS', 'MLWS', 'MHWN', 'MLWN', 'year', 'year_start', 'set', 'M4', 'O1',
       'K2', 'K1', 'P1', 'N2', 'M6', 'MU2', 'L2', '2MS6', 'MS4', 'NU2', 'SA'],
      dtype='object')

In [8]:
#df = df[np.abs(df['z0']) < 1e2]
#df = df[np.abs(df['M2']) < 1e2]
df = df[df['year'].between(1945, 2005)]

In [9]:
names = [
    'Delfzijl',
    'Harlingen',
    'Den Helder',
    'IJmuiden',
    'Hoek van Holland',
    'Vlissingen'
]

In [10]:
df = df[df['set'] == 'Ftested3']

#### Calibrate per model

In [11]:
param = pd.DataFrame(columns = [
    'name',
    'var',
    'model',
    'intercept',
    'slope',
    'acceleration',
    'A_885',
    'A_1861',
    'Rsqadj'], dtype = float)

In [12]:
vars = ['z0', 'MHWS', 'MLWS']

In [13]:
tmp = pd.DataFrame(columns = ['name'])

namestore = []
varstore = []
modelstore = []
intercept = []
slope = []
Rsqadj = []

for vr in vars:
    for nm in names:
        if nm == 'Delfzijl':
            if vr != 'z0':
                continue

        data = df[df['naam'] == nm]
        res = gf.fit_with_uncert(
            mdl.model1,
            data['year'],
            data[vr]
        )

        p = res[0]
        
        namestore.append(nm)
        varstore.append(vr)
        modelstore.append('model1')
        intercept.append(p[0])
        slope.append(p[1])
        Rsqadj.append(res[5])


tmp['name'] = namestore
tmp['var'] = varstore
tmp['model'] = modelstore
tmp['intercept'] = intercept
tmp['slope'] = slope
tmp['Rsqadj'] = Rsqadj

param = pd.concat([param, tmp])

In [14]:
tmp = pd.DataFrame(columns = ['name'])

namestore = []
varstore = []
modelstore = []
intercept = []
slope = []
acceleration = []
Rsqadj = []

for vr in vars:
    for nm in names:
        if nm == 'Delfzijl':
            if vr != 'z0':
                continue

        data = df[df['naam'] == nm]
        res = gf.fit_with_uncert(
            mdl.model2,
            data['year'],
            data[vr]
        )

        p = res[0]

        namestore.append(nm)
        varstore.append(vr)
        modelstore.append('model2')
        intercept.append(p[0])
        slope.append(p[1])
        acceleration.append(p[2])
        Rsqadj.append(res[5])
        

tmp['name'] = namestore
tmp['var'] = varstore
tmp['model'] = modelstore
tmp['intercept'] = intercept
tmp['slope'] = slope
tmp['acceleration'] = acceleration
tmp['Rsqadj'] = Rsqadj

param = pd.concat([param, tmp])

In [15]:
tmp = pd.DataFrame(columns = ['name'])

namestore = []
varstore = []
modelstore = []
intercept = []
slope = []
acceleration = []
A_885 = []
A_1861 = []
Rsqadj = []

for vr in vars:
    for nm in names:
        if nm == 'Delfzijl':
            if vr != 'z0':
                continue

        data = df[df['naam'] == nm]
        res = gf.fit_with_uncert(
            mdl.model3,
            data['year'],
            data[vr]
        )

        p = res[0]

        namestore.append(nm)
        varstore.append(vr)
        modelstore.append('model3')
        intercept.append(p[0])
        slope.append(p[1])
        #acceleration.append(p[2])
        A_885.append(np.sqrt(p[2] ** 2 + p[3] ** 2))
        A_1861.append(np.sqrt(p[4] ** 2 + p[5] ** 2))
        Rsqadj.append(res[5])
        

tmp['name'] = namestore
tmp['var'] = varstore
tmp['model'] = modelstore
tmp['intercept'] = intercept
tmp['slope'] = slope
#tmp['acceleration'] = acceleration
tmp['A_885'] = A_885
tmp['A_1861'] = A_1861
tmp['Rsqadj'] = Rsqadj

param = pd.concat([param, tmp])

In [16]:
tmp = pd.DataFrame()

namestore = []
varstore = []
modelstore = []
intercept = []
slope = []
acceleration = []
A_885 = []
A_1861 = []
Rsqadj = []

for vr in vars:
    for nm in names:
        if nm == 'Delfzijl':
            if vr != 'z0':
                continue

        data = df[df['naam'] == nm]
        res = gf.fit_with_uncert(
            mdl.model4,
            data['year'],
            data[vr]
        )

        p = res[0]

        namestore.append(nm)
        varstore.append(vr)
        modelstore.append('model4')
        intercept.append(p[0])
        slope.append(p[1])
        acceleration.append(p[2])
        A_885.append(np.sqrt(p[2] ** 2 + p[3] ** 2))
        A_1861.append(np.sqrt(p[4] ** 2 + p[5] ** 2))
        Rsqadj.append(res[5])
        

tmp['name'] = namestore
tmp['var'] = varstore
tmp['model'] = modelstore
tmp['intercept'] = intercept
tmp['slope'] = slope
tmp['acceleration'] = acceleration
tmp['A_885'] = A_885
tmp['A_1861'] = A_1861
tmp['Rsqadj'] = Rsqadj

param = pd.concat([param, tmp])

In [17]:
# Set units (length mm and time centuries)
param['intercept'] = (1000 * param['intercept']).round()
param['slope'] = (1e5 * param['slope']).round()
param['acceleration'] = (1e7 * param['acceleration']).round()
param['A_885'] = (1e3 * param['A_885']).round()
param['A_1861'] = (1e3 * param['A_1861']).round()
param['Rsqadj'] = (1e2 * param['Rsqadj']).round()

In [18]:
param.sort_values(by = ['name', 'model'], inplace = True)

In [19]:
param[param['var'] == 'z0'].to_excel(r'../results/fitted_models_z0.xlsx')
param[param['var'] == 'MHWS'].to_excel(r'../results/fitted_models_MHWS.xlsx')
param[param['var'] == 'MLWS'].to_excel(r'../results/fitted_models_MLWS.xlsx')


#### End script
Prepared by HVEC lab, 2022